In [ ]:
import time
import datetime
import dateutil.relativedelta
import json
import requests

import pandas as pd

#### Write function which serves for sending HTTP request to get API data

In [ ]:
def send_request(str_ts):
    try:
        print('url: ', "https://discover.energy/api/XXX"+str_ts)
        response = requests.get(
            url="https://discover.energy/api/XXX"+str_ts,
            headers={
                "Authorization": "XXX",
            },
        )
        return response.content
    except requests.exceptions.RequestException:
        print('HTTP Request failed')

In [ ]:
ts = (int((datetime.datetime.now() 
           - dateutil.relativedelta.relativedelta(months=60)).timestamp()))

df = pd.DataFrame(json.loads(send_request(str(ts)))['data'])

df['date'] = pd.to_datetime(df['created_at'], infer_datetime_format=True).dt.to_period('d')

df['user_type'] = df['user'].apply(lambda x: str(type(x))[-6:-2])

df = df[df['user_type'] == 'dict'].copy()
df['user_email'] = df['user'].apply(lambda x: x['email'])

df['domain'] = df['user_email'].apply(lambda x: x.split('@')[1])

internal_domains = ['heaconnect.com', 'tapmates.com', 'advisor.energy', 'tracktivity.com']
group_metrics = ['reports', 'assets', 'metrics']

df = df[~df['domain'].isin(internal_domains)]
df = df[df['group'].isin(group_metrics)]

In [ ]:
clicks_analysis = df[['date', 'group', 'domain', 'user_email']].reset_index(drop=True)
unique_users_analysis = (clicks_analysis[['date', 'domain', 'user_email']]
                         .drop_duplicates().reset_index(drop=True))

In [ ]:
clicks_analysis.to_csv('clicks_analysis.csv')
unique_users_analysis.to_csv('unique_users_analysis.csv')

In [ ]:
advisor_df = df.copy()
advisor_df['created_at'] = pd.to_datetime(advisor_df['created_at'], infer_datetime_format=True)

advisor_df['entity_type'] = advisor_df['related_entity'].apply(lambda x: str(type(x))[-6:-2])
advisor_df = advisor_df[advisor_df['entity_type'] == "dict"].copy()
advisor_df['name'] = advisor_df['related_entity'].apply(lambda x: x['name'])
advisor_df['report_class'] = advisor_df['related_entity'].apply(lambda x: x['class']) 
advisor_df = advisor_df[advisor_df['report_class'] == 'ReportVersion'].copy()
advisor_df = advisor_df.drop(['data', 'date', 'ip_address', 'ip_location', 'user', 
                              'user_type', 'report_class', 'entity_type', 
                              'related_entity', 'type'], axis=1)

In [ ]:
fd = (advisor_df[['name', 'created_at']]
      .groupby('name')
      .min()
      .reset_index()
      .rename(columns={'created_at':'first_date'}))

In [ ]:
advisor_df = advisor_df.merge(fd, on='name', how='inner')
advisor_df['dt'] = advisor_df['created_at'] - advisor_df['first_date']

In [ ]:
advisor_df['f24h'] = 0
advisor_df.loc[((advisor_df['dt'] <= pd.to_timedelta('1 days 00:00:00'))), 'f24h'] = 1
advisor_df['5d'] = 0
advisor_df.loc[((advisor_df['dt'] <= pd.to_timedelta('5 days 00:00:00'))), '5d'] = 1
advisor_df['30d'] = 0
advisor_df.loc[((advisor_df['dt'] <= pd.to_timedelta('30 days 00:00:00'))), '30d'] = 1

In [ ]:
advisor_df.to_csv('advisor_review.csv')